In [1]:
from flextag.managers.query import QueryManager
from flextag.managers.data import DataManager
from flextag.core.query.provider import QueryProvider
from flextag.core.parser.provider import ParserProvider

# Setup with more verbose output
query_manager = QueryManager()
data_manager = DataManager()

query_provider = QueryProvider()
parser_provider = ParserProvider()

query_manager.register("query_parsers", "default", query_provider)
data_manager.register("parsers", "default", parser_provider)
data_manager.connect(query_manager)

print("Setup complete\n")

DEBUG:flextag.managers.base:Initialized search_manager
DEBUG:flextag.managers.base:search_manager: Subscribed to block_parsed
DEBUG:flextag.managers.base:Initialized data_manager
DEBUG:flextag.managers.base:data_manager: Created registry 'parsers'
DEBUG:flextag.managers.base:data_manager: Created registry 'content_handlers'
DEBUG:flextag.managers.base:search_manager: Created registry 'query_parsers'
DEBUG:flextag.managers.base:search_manager: Registered 'default' in 'query_parsers'
DEBUG:flextag.managers.base:data_manager: Registered 'default' in 'parsers'
DEBUG:flextag.managers.base:data_manager: Connected to search_manager


Setup complete


In [2]:
# Test 1: Verify parser is working
test_data = """
[[DATA:block1 #test .path.to.test]]
Content
[[/DATA]]
"""

# Parse and show the blocks
blocks = list(parser_provider.parse_string(test_data))
print("Parsed blocks:")
for block in blocks:
    print(f"\nBlock details:")
    print(f"ID: {block.id}")
    print(f"Tags: {block.tags}")
    print(f"Paths: {block.paths}")
    print(f"Content lines: {block.content_start}-{block.content_end}")

Parsed blocks:

Block details:
ID: block1
Tags: ['test']
Paths: ['path.to.test']
Content lines: 3-3


In [1]:
# Test 2: Verify DuckDB table
print("DuckDB table contents before parsing:")
print(query_manager.conn.execute("SELECT * FROM blocks").fetchall())

# Parse data to populate DuckDB
data_manager.parse_string(test_data)

print("\nDuckDB table contents after parsing:")
print(query_manager.conn.execute("SELECT * FROM blocks").fetchall())


DuckDB table contents before parsing:


NameError: name 'query_manager' is not defined

In [ ]:
# Test 3: Verify query conversion
test_queries = [
    ":block1",      # ID query
    "#test",        # Tag query
    ".path.to.test" # Path query
]

print("Query conversion test:")
for query in test_queries:
    sql = query_provider.to_sql(query)
    print(f"\nQuery: {query}")
    print(f"SQL: {sql}")
    
    # Execute and show results
    results = query_manager.search(query)
    print(f"Results: {results}")

In [ ]:
# Test 4: Direct SQL query to verify data
print("Direct SQL queries:")

# Check block IDs
print("\nBlock IDs:")
print(query_manager.conn.execute("SELECT block_id FROM blocks").fetchall())

# Check tags
print("\nTags:")
print(query_manager.conn.execute("SELECT tags FROM blocks").fetchall())

# Check paths
print("\nPaths:")
print(query_manager.conn.execute("SELECT paths FROM blocks").fetchall())

# Full row dump
print("\nFull rows:")
print(query_manager.conn.execute("SELECT * FROM blocks").fetchall())